# Zillow Lead Scoring

## Imports & Setup

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, LongType, DoubleType, TimestampType, StructType, StructField
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, Bucketizer, VectorAssembler, StandardScaler, OneHotEncoder, Imputer
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier

In [2]:
spark = SparkSession.builder.appName('leadscoring').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/08 13:12:00 WARN Utils: Your hostname, Jordans-MacBook-Pro-2.local, resolves to a loopback address: 127.0.0.1; using 192.168.10.76 instead (on interface en0)
25/11/08 13:12:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/08 13:12:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sc = spark.sparkContext # Create spark context

## Reading the Dataset

In [4]:
path = '/Users/jordan/Code/ML/realtime-lead-scoring/data/raw/'

In [5]:
zillowSchema = StructType([
    StructField('ProspectID', StringType(), True),
    StructField('LeadNumber', LongType(), True),
    StructField('LeadCaptureChannel', StringType(), True),
    StructField('ReferralSource', StringType(), True),
    StructField('OptOutEmail', StringType(), True),
    StructField('OptOutCall', StringType(), True),
    StructField('ContactedAgent', LongType(), True),
    StructField('TotalVisits', LongType(), True),
    StructField('TotalBrowsingTime', LongType(), True),
    StructField('AvgListingsViewedPerSession', DoubleType(), True),
    StructField('LastAction', StringType(), True),
    StructField('Country', StringType(), True),
    StructField('Specialization', StringType(), True),
    StructField('How did you hear about Zillow', StringType(), True),
    StructField('What is your current occupation', StringType(), True),
    StructField('What matters most to you in choosing this house', StringType(), True),
    StructField('Search', StringType(), True),
    StructField('Magazine', StringType(), True),
    StructField('Newspaper Article', StringType(), True),
    StructField('Zillow Forums', StringType(), True),
    StructField('Newspaper', StringType(), True),
    StructField('Digital Advertisement', StringType(), True),
    StructField('Through Recommendations', StringType(), True),
    StructField('Receive More Updates About Our Houses', StringType(), True),
    StructField('LeadStatusTag', StringType(), True),
    StructField('LeadQuality', StringType(), True),
    StructField('Update me on Zillow Content', StringType(), True),
    StructField('Get updates on DM Content', StringType(), True),
    StructField('LeadProfile', StringType(), True),
    StructField('City', StringType(), True),
    StructField('Asymmetric_Activity_Index', StringType(), True),
    StructField('Asymmetric_Profile_Index', StringType(), True),
    StructField('Asymmetric_Activity_Score', LongType(), True),
    StructField('Asymmetric_Profile_Score', LongType(), True),
    StructField('I agree to pay the amount through cheque', StringType(), True),
    StructField('a free copy of Mastering The Interview', StringType(), True),
    StructField('FinalEngagementAction', StringType(), True)
])

In [6]:
df = spark.read.option("dropMalformed", True).option("ignoreLeadingWhiteSpace", True).csv(path, header=True, schema=zillowSchema)

In [7]:
df = df.select(
    [
        "ContactedAgent", "TotalVisits", "TotalBrowsingTime", "AvgListingsViewedPerSession",
        "LeadCaptureChannel", "ReferralSource", "LastAction", "FinalEngagementAction",
        "City", "Country", "LeadStatusTag", "ProspectID", "LeadNumber"
    ]
)

In [8]:
df.show()

+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+------------------+-------+--------------------+--------------------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|  LeadCaptureChannel|ReferralSource|          LastAction|FinalEngagementAction|              City|Country|       LeadStatusTag|          ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+------------------+-------+--------------------+--------------------+----------+
|             0|          0|                0|                        0.0|                 API|    Olark Chat|Page Visited on W...|             Modified|            Select|   NULL|Interested in oth...|7927b2df-8bba-4d2...|    660737|
|             0|          5|              674|                  

25/11/08 13:12:23 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [9]:
# Split data into training and testing
train, test = df.randomSplit([0.8, 0.2])

In [10]:
train.count(), test.count()

25/11/08 13:12:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv
25/11/08 13:12:50 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, Tot

(7434, 1806)

## Data Exploration

In [31]:
# Class balance
train.groupBy("ContactedAgent").count().show()

+--------------+-----+
|ContactedAgent|count|
+--------------+-----+
|             0| 4582|
|             1| 2852|
+--------------+-----+



25/11/08 13:19:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


The positive class (1) indicating a user contacted the agent has less representation in the dataset. It has about 38% of the total number of samples. This makes sense because it's less likely for a user to contact an agent in general. 

In [36]:
# Distribution of numerical features
numerical_features = ["TotalVisits", "TotalBrowsingTime", "AvgListingsViewedPerSession"]
for feature in numerical_features:
    train.select(feature).describe().show()

25/11/08 13:26:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv
25/11/08 13:26:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, Tot

+-------+------------------+
|summary|       TotalVisits|
+-------+------------------+
|  count|              7327|
|   mean|3.4528456394158593|
| stddev| 5.150896094051949|
|    min|                 0|
|    max|               251|
+-------+------------------+

+-------+------------------+
|summary| TotalBrowsingTime|
+-------+------------------+
|  count|              7434|
|   mean|487.58165186978744|
| stddev| 548.7222601518283|
|    min|                 0|
|    max|              2272|
+-------+------------------+

+-------+---------------------------+
|summary|AvgListingsViewedPerSession|
+-------+---------------------------+
|  count|                       7327|
|   mean|          2.361759246622083|
| stddev|         2.1889505618644107|
|    min|                        0.0|
|    max|                       55.0|
+-------+---------------------------+



In [38]:
# Evaluate missing values in each column
from pyspark.sql.functions import col, sum as spark_sum
missing_values = train.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in train.columns])
missing_values.show()

+--------------+-----------+-----------------+---------------------------+------------------+--------------+----------+---------------------+----+-------+-------------+----------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|LeadCaptureChannel|ReferralSource|LastAction|FinalEngagementAction|City|Country|LeadStatusTag|ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+----------+---------------------+----+-------+-------------+----------+----------+
|             0|        107|                0|                        107|                 0|            27|        80|                    0|1153|   1997|         2720|         0|         0|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+----------+---------------------+----+-------+-------------+----------+----------+



25/11/08 13:28:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [41]:
# Check the count of each unique value in the country column
train.groupBy("Country").count().show(500)

+--------------------+-----+
|             Country|count|
+--------------------+-----+
|              Sweden|    1|
|         Philippines|    1|
|           Singapore|   23|
|            Malaysia|    1|
|             Germany|    3|
|              France|    5|
|           Sri Lanka|    1|
|             unknown|    4|
| Asia/Pacific Region|    2|
|                NULL| 1997|
|             Belgium|    2|
|               Qatar|    9|
|               Ghana|    2|
|               India| 5197|
|       United States|   55|
|               China|    2|
|              Kuwait|    3|
|             Nigeria|    3|
|               Italy|    2|
|             Denmark|    1|
|          Bangladesh|    2|
|           Hong Kong|    6|
|                Oman|    6|
|           Indonesia|    1|
|             Liberia|    1|
|        Saudi Arabia|   18|
|              Uganda|    2|
|United Arab Emirates|   43|
|              Canada|    4|
|               Kenya|    1|
|            Tanzania|    1|
|           Au

25/11/08 13:31:24 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [42]:
# Check the count of each unique value in the city column
train.groupBy("City").count().show(500)

+--------------------+-----+
|                City|count|
+--------------------+-----+
|      Tier II Cities|   60|
|                NULL| 1153|
|              Mumbai| 2563|
|  Other Metro Cities|  308|
|        Other Cities|  551|
|              Select| 1835|
|Other Cities of M...|  359|
|   Thane & Outskirts|  605|
+--------------------+-----+



25/11/08 13:32:15 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [43]:
# Check the unique value counts for LeadStatusTag
train.groupBy("LeadStatusTag").count().show(500)

+--------------------+-----+
|       LeadStatusTag|count|
+--------------------+-----+
|  wrong number given|   39|
|University not re...|    1|
|     Lateral student|    3|
|Want to take admi...|    5|
|Not doing further...|  118|
|                NULL| 2720|
|      Lost to Others|    7|
|Shall take in the...|    2|
|  in touch with EINS|    9|
|Diploma holder (N...|   49|
|                Busy|  148|
|In confusion whet...|    5|
|Graduation in pro...|   96|
|Will revert after...| 1650|
|             Ringing|  958|
|        switched off|  189|
|  Closed by Horizzon|  283|
|   Already a student|  379|
|          opp hangup|   29|
| number not provided|   23|
|Interested  in fu...|  100|
|      Still Thinking|    5|
|Interested in oth...|  417|
|Interested in Nex...|    3|
|      invalid number|   63|
|        Lost to EINS|  133|
+--------------------+-----+



25/11/08 13:33:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


City, country, and lead status tag have quite a lot of missing values. It may be worth dropping city and country. While they could have relevance to the target, it appears that most users where from India. And the city column values show that most users were assigned to mumbai or "Select", which seems like it means the user didn't specify. The lead status tag could be useful. I think we could consider NULL as a "No Status" tag. 

In [45]:
# Change NULL LeadStatusTag to "No Status"
# Spark dataframes do not support fillna on specific columns with string values directly, so we use when/otherwise
from pyspark.sql.functions import when
df = df.withColumn("LeadStatusTag", when(col("LeadStatusTag").isNull(), "No Status").otherwise(col("LeadStatusTag")))

In [48]:
# Drop City and Country due to high missing values and low relevance
df = df.select([col for col in df.columns if col not in ["City", "Country"]])

In [49]:
df.show(5)

+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+--------------------+--------------------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|  LeadCaptureChannel|ReferralSource|          LastAction|FinalEngagementAction|       LeadStatusTag|          ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+--------------------+--------------------+----------+
|             0|          0|                0|                        0.0|                 API|    Olark Chat|Page Visited on W...|             Modified|Interested in oth...|7927b2df-8bba-4d2...|    660737|
|             0|          5|              674|                        2.5|                 API|Organic Search|        Email Opened|         Email Opened|             Ringin

25/11/08 13:38:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Tags, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, LeadStatusTag, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


Now, for the last action, we could just replace the null values with "No Action", but it might be worth comparing last action to the final engagement action to see if we really need both. 

In [47]:
# Compare LastAction to FinalEngagementAction to see how often they match
df.filter(col("LastAction") == col("FinalEngagementAction")).count(), df.count()

25/11/08 13:37:53 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Last Activity, Last Notable Activity
 Schema: LastAction, FinalEngagementAction
Expected: LastAction but found: Last Activity
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


(5833, 9240)

Over half of the last action values are equal to the final engagement values. 

In [50]:
# Check unique values in LastAction and FinalEngagementAction
df.select("LastAction").distinct().show(100)
df.select("FinalEngagementAction").distinct().show(100)

+--------------------+
|          LastAction|
+--------------------+
|Form Submitted on...|
|        Email Opened|
|Resubscribed to e...|
|         Unreachable|
|Visited Booth in ...|
|View in browser l...|
|  Email Link Clicked|
|   Converted to Lead|
|Olark Chat Conver...|
|            SMS Sent|
|        Unsubscribed|
|Page Visited on W...|
|   Email Marked Spam|
|       Email Bounced|
|      Email Received|
|Had a Phone Conve...|
|  Approached upfront|
|                NULL|
+--------------------+

+---------------------+
|FinalEngagementAction|
+---------------------+
| Form Submitted on...|
|         Email Opened|
| Resubscribed to e...|
|          Unreachable|
| View in browser l...|
|   Email Link Clicked|
| Olark Chat Conver...|
|             SMS Sent|
|             Modified|
|         Unsubscribed|
| Page Visited on W...|
|    Email Marked Spam|
|        Email Bounced|
|       Email Received|
| Had a Phone Conve...|
|   Approached upfront|
+---------------------+



25/11/08 13:40:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Last Activity
 Schema: LastAction
Expected: LastAction but found: Last Activity
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv
25/11/08 13:40:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Last Notable Activity
 Schema: FinalEngagementAction
Expected: FinalEngagementAction but found: Last Notable Activity
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


They have a few differences, but are largely the same values. I think we should keep the final engagement values and remove the last action column. 

In [51]:
df = df.select([col for col in df.columns if col != "LastAction"])

In [52]:
df.show(5)

+--------------+-----------+-----------------+---------------------------+--------------------+--------------+---------------------+--------------------+--------------------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|  LeadCaptureChannel|ReferralSource|FinalEngagementAction|       LeadStatusTag|          ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+--------------------+--------------+---------------------+--------------------+--------------------+----------+
|             0|          0|                0|                        0.0|                 API|    Olark Chat|             Modified|Interested in oth...|7927b2df-8bba-4d2...|    660737|
|             0|          5|              674|                        2.5|                 API|Organic Search|         Email Opened|             Ringing|2a272436-5132-413...|    660728|
|             1|          2|             1532|                        

25/11/08 13:41:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Tags, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LeadStatusTag, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [53]:
df.groupBy("ReferralSource").count().show(500)

+-----------------+-----+
|   ReferralSource|count|
+-----------------+-----+
|   youtubechannel|    1|
|             bing|    6|
|       Olark Chat| 1755|
|        Live Chat|    2|
|             NULL|   36|
|Pay per Click Ads|    1|
|    Press_Release|    2|
|          testone|    1|
|     Social Media|    2|
|   Referral Sites|  125|
|        Reference|  534|
|          WeLearn|    1|
|           Google| 2868|
|   Direct Traffic| 2543|
|             blog|    1|
| Welingak Website|  142|
| welearnblog_Home|    1|
|   Organic Search| 1154|
|           NC_EDM|    1|
|       Click2call|    4|
|         Facebook|   55|
|           google|    5|
+-----------------+-----+



25/11/08 13:44:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Lead Source
 Schema: ReferralSource
Expected: ReferralSource but found: Lead Source
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


Finally, we should replace any Null values in the Referal Source column to say something like "None" or "Unknown". 

In [54]:
# Update referral source nulls to "Unknown"
df = df.withColumn("ReferralSource", when(col("ReferralSource").isNull(), "Unknown").otherwise(col("ReferralSource")))

In [55]:
# Check for missing values now
missing_values = df.select([spark_sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
missing_values.show()

+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+-------------+----------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|LeadCaptureChannel|ReferralSource|FinalEngagementAction|LeadStatusTag|ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+-------------+----------+----------+
|             0|        137|                0|                        137|                 0|             0|                    0|            0|         0|         0|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+-------------+----------+----------+



25/11/08 13:44:54 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Tags, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LeadStatusTag, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


We have handled the categorical missing values, and the numeric ones will be imputed in the ml pipeline. 

In [57]:
import plotly.express as px
fig = px.box(train.toPandas(), x='ContactedAgent', y='TotalBrowsingTime', title='Total Browsing Time by Contacted Agent Status')
fig.show()

25/11/08 13:46:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


That makes sense, users with longer browsing times are more likely to contact the agent. 

In [59]:
df.show(1)

+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+--------------------+--------------------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|LeadCaptureChannel|ReferralSource|FinalEngagementAction|       LeadStatusTag|          ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+--------------------+--------------------+----------+
|             0|          0|                0|                        0.0|               API|    Olark Chat|             Modified|Interested in oth...|7927b2df-8bba-4d2...|    660737|
+--------------+-----------+-----------------+---------------------------+------------------+--------------+---------------------+--------------------+--------------------+----------+
only showing top 1 row


25/11/08 13:48:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Tags, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LeadStatusTag, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [ ]:
# Distribution of numerical features
numerical_features = ["TotalVisits", "TotalBrowsingTime", "AvgListingsViewedPerSession"]
for feature in numerical_features:
    fig = px.histogram(train.toPandas(), x=feature, title=f'Distribution of {feature}')
    fig.show()

25/11/08 14:22:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


25/11/08 14:22:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


25/11/08 14:22:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


25/11/08 16:16:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 956602 ms exceeds timeout 120000 ms
25/11/08 16:16:49 WARN SparkContext: Killing executors is not supported by current scheduler.
25/11/08 16:16:50 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

## Building the Pipeline Model

In [11]:
pipeline = Pipeline(stages=[
    Imputer(inputCols=['TotalVisits', 'TotalBrowsingTime', 'AvgListingsViewedPerSession'], outputCols=['TotalVisits', 'TotalBrowsingTime', 'AvgListingsViewedPerSession']),
    Bucketizer(inputCol="TotalVisits", outputCol="TotalVisits_bucket", splits=[0, 10, 20, 50, 75, 100, 150, 200, 300, float('Inf')]),
    StringIndexer(inputCol='LeadCaptureChannel', outputCol='LeadCaptureChannel_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='LeadCaptureChannel_index', outputCol='LeadCaptureChannel_ohe'),
    StringIndexer(inputCol='ReferralSource', outputCol='ReferralSource_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='ReferralSource_index', outputCol='ReferralSource_ohe'),
    StringIndexer(inputCol='LastAction', outputCol='LastAction_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='LastAction_index', outputCol='LastAction_ohe'),
    StringIndexer(inputCol='FinalEngagementAction', outputCol='FinalEngagementAction_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='FinalEngagementAction_index', outputCol='FinalEngagementAction_ohe'),
    StringIndexer(inputCol='City', outputCol='City_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='City_index', outputCol='City_ohe'),
    StringIndexer(inputCol='Country', outputCol='Country_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='Country_index', outputCol='Country_ohe'),
    StringIndexer(inputCol='LeadStatusTag', outputCol='LeadStatusTag_index', handleInvalid="keep"),
    OneHotEncoder(inputCol='LeadStatusTag_index', outputCol='LeadStatusTag_ohe'),
    VectorAssembler(inputCols=["TotalVisits_bucket", "TotalBrowsingTime", "AvgListingsViewedPerSession", "LeadCaptureChannel_ohe", "ReferralSource_ohe", "LastAction_ohe", "City_ohe", "Country_ohe", "LeadStatusTag_ohe"], outputCol='features'),
    RandomForestClassifier(featuresCol='features', labelCol='ContactedAgent')
])

In [12]:
leadmodel = pipeline.fit(train)

25/11/08 13:13:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv
25/11/08 13:13:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, Tot

In [13]:
pred = leadmodel.transform(test)

In [14]:
# Test Accuracy
pred.filter(pred.ContactedAgent == pred.prediction).count() / pred.count()

25/11/08 13:13:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv
25/11/08 13:13:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, Tot

0.8654485049833887

In [15]:
# display the id, probability and prediction data (just the first 20 lines are fine)
pred.select('ProspectID', 'probability', 'prediction', "ContactedAgent").show(20)

+--------------------+--------------------+----------+--------------+
|          ProspectID|         probability|prediction|ContactedAgent|
+--------------------+--------------------+----------+--------------+
|6be73d7d-525d-11e...|[0.79688892366402...|       0.0|             0|
|498b9fe5-4e52-11e...|[0.55725402978900...|       0.0|             0|
|498ba7d0-4e52-11e...|[0.55725402978900...|       0.0|             0|
|f8c8282b-33dd-455...|[0.80462539703303...|       0.0|             0|
|498bc13b-4e52-11e...|[0.74542054844436...|       0.0|             0|
|5e748f60-2b1c-479...|[0.26668235252672...|       1.0|             0|
|ce400e14-966f-44b...|[0.17580445666315...|       1.0|             0|
|6be73f79-525d-11e...|[0.75766836413399...|       0.0|             0|
|f4f486ac-70f9-4d6...|[0.78130911521717...|       0.0|             0|
|13f7d9cd-a47c-402...|[0.81170140374731...|       0.0|             0|
|1520480f-4226-4fe...|[0.81170140374731...|       0.0|             0|
|6f9bdf8d-21ff-461..

25/11/08 13:13:48 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


## Saving Test Data to Separate Files for Streaming

In [16]:
test = test.repartition(100)
test.rdd.getNumPartitions()

25/11/08 13:14:09 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


100

In [18]:
path = '/Users/jordan/Code/ML/realtime-lead-scoring/data/'

In [ ]:
test.write.format("csv").option("header", "true").save(path + "stream/user_events")

## Streaming

In [25]:
# Source
# Files
sourceStream = spark.readStream.format("parquet").option("header", "true").option("maxFilesPerTrigger", 1).schema(test.schema).load(path + "stream/user_events/")

# Query
query = leadmodel.transform(sourceStream).select("ProspectID", "ContactedAgent", "prediction", "probability")

# Sink
streamSink = query.writeStream.outputMode("update").format("memory").queryName("leads").trigger(processingTime='5 seconds').start()

25/11/08 13:17:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/qt/4yhgm0j53w96n6z_j5zct2k40000gn/T/temporary-662731ec-1add-4985-8c32-ca3e78ed399c. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/08 13:17:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [26]:
df.show()

+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+------------------+-------+--------------------+--------------------+----------+
|ContactedAgent|TotalVisits|TotalBrowsingTime|AvgListingsViewedPerSession|  LeadCaptureChannel|ReferralSource|          LastAction|FinalEngagementAction|              City|Country|       LeadStatusTag|          ProspectID|LeadNumber|
+--------------+-----------+-----------------+---------------------------+--------------------+--------------+--------------------+---------------------+------------------+-------+--------------------+--------------------+----------+
|             0|          0|                0|                        0.0|                 API|    Olark Chat|Page Visited on W...|             Modified|            Select|   NULL|Interested in oth...|7927b2df-8bba-4d2...|    660737|
|             0|          5|              674|                  

25/11/08 13:17:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Prospect ID, Lead Number, Lead Origin, Lead Source, Converted, TotalVisits, Total Time Spent on Website, Page Views Per Visit, Last Activity, Country, Tags, City, Last Notable Activity
 Schema: ProspectID, LeadNumber, LeadCaptureChannel, ReferralSource, ContactedAgent, TotalVisits, TotalBrowsingTime, AvgListingsViewedPerSession, LastAction, Country, LeadStatusTag, City, FinalEngagementAction
Expected: ProspectID but found: Prospect ID
CSV file: file:///Users/jordan/Code/ML/realtime-lead-scoring/data/raw/Lead%20Scoring.csv


In [29]:
spark.sql("select * from leads").show(truncate=False)

+------------------------------------+--------------+----------+----------------------------------------+
|ProspectID                          |ContactedAgent|prediction|probability                             |
+------------------------------------+--------------+----------+----------------------------------------+
|055df508-28b1-42ee-8fc3-3816d536af12|0             |0.0       |[0.819131296251421,0.18086870374857905] |
|abcbb1cd-ca33-4213-88d1-eaef1c468cb3|0             |0.0       |[0.7329915333544793,0.26700846664552075]|
|2f93b5df-7a90-4cc4-8790-c50907696f65|0             |0.0       |[0.7476042048202918,0.2523957951797083] |
|4cb1fc3c-1fa3-4a8c-b3fc-24d5a711df88|0             |0.0       |[0.7502400352656777,0.24975996473432227]|
|d7160279-3aef-4c1d-875b-5c08866330de|0             |0.0       |[0.7368582278219317,0.26314177217806833]|
|5fd4ea2d-97fa-4022-b505-5f8dbd40fe41|1             |0.0       |[0.6398048327762564,0.3601951672237437] |
|24e4322e-57b7-4919-ab07-7d01c66a19a8|0       

In [30]:
for s in spark.streams.active:
    if s.name == "leads":
        s.stop()

25/11/08 13:18:02 WARN DAGScheduler: Failed to cancel job group d7b8d440-5325-4c6c-9b05-28253c66331e. Cannot find active jobs for it.
25/11/08 13:18:02 WARN DAGScheduler: Failed to cancel job group d7b8d440-5325-4c6c-9b05-28253c66331e. Cannot find active jobs for it.
